In [128]:
import matplotlib.pyplot as plt
import numpy as np

from mesa import Agent, Model
from mesa.batchrunner import BatchRunnerMP
from mesa.datacollection import DataCollector
from mesa.space import Grid
from mesa.time import RandomActivation

In [52]:
COLORS = {
    TreeAgent.STATE_GOOD: 1,
    TreeAgent.STATE_FIRE: 2,
    TreeAgent.STATE_BURNT: 3
}


def plot_grid(grid):
    grid_translated = []
    for row in grid:
        
        row_translated = []
        for el in row:
            if isinstance(el, TreeAgent):
                row_translated.append(COLORS[el.state])
            else:
                row_translated.append(0)
        grid_translated.append(row_translated)
    
    return grid_translated

In [126]:
class TreeAgent(Agent):
    STATE_GOOD = 'Good'
    STATE_FIRE = 'Fire'
    STATE_BURNT = 'Burnt'
    
    def __init__(self, pos, model):
        super().__init__(pos, model)
        self.pos = pos
        self.state = self.STATE_GOOD
        self.state_future = None
        
    def apply(self):
        if self.state_future:
            self.state = self.state_future
            self.state_future = None
        
    def is_good(self):
        return self.state == self.STATE_GOOD
        
    def is_on_fire(self):
        return self.state == self.STATE_FIRE
        
    def set_fire(self):
        self.state_future = self.STATE_FIRE
        
    def set_burnt(self):
        self.state_future = self.STATE_BURNT
    
    def step(self):
        if self.is_on_fire():
            for neighbor in self.model.grid.neighbor_iter(self.pos):
                if neighbor.is_good():
                    neighbor.set_fire()
            self.set_burnt()

        
class ForestModel(Model):
    
    def __init__(self, grid_shape, p):
        self.grid = Grid(grid_shape[0], grid_shape[1], torus = False)
        self.schedule = RandomActivation(self)
        self.running = True
        
        for a, x, y in self.grid.coord_iter():
            
            if self.random.random() > p:
                agent = TreeAgent((x, y), self)
                
                if x == 0:
                    agent.set_fire()
                    agent.apply()
                
                self.schedule.add(agent)
                self.grid.place_agent(agent, (x, y))

        
    def step(self):
        self.schedule.step()
        self.apply_step()
        self.running = self.is_running()
        
    def apply_step(self):
        for agent in self.schedule.agents:
            agent.apply()
        
    def is_running(self):
        return any([agent.state == TreeAgent.STATE_FIRE for agent in self.schedule.agents])
    

In [120]:
def compute_is_burnt(model):
    return 1

def compute_max_cluster_size(model):
    return 1

In [129]:
params_variable = {
    "p": np.arange(0, 1.02, 0.5),
    "grid_shape": [(20,20), (50,50), (100, 100)]
}

batch_runner = BatchRunnerMP(
    ForestModel,
    nr_processes = 2,
    variable_parameters = params_variable,
    iterations = 5,
    model_reporters={
        "Burnt": compute_is_burnt,
        "Max cluster size": compute_max_cluster_size
    }
)

MPSupport: BatchRunnerMP depends on pathos, which is either not installed, or the path can not be found. 

In [124]:
batch_runner.run_all()

4it [00:00, 23.97it/s]

(20, 20)
(20, 20)
(20, 20)
(20, 20)
(20, 20)
(50, 50)


7it [00:00, 10.83it/s]

(50, 50)
(50, 50)


9it [00:00,  7.45it/s]

...

(100, 100)


29it [00:10,  1.60it/s]

(100, 100)


45it [00:11,  3.91it/s]

(20, 20)
(20, 20)
(20, 20)
(20, 20)
(20, 20)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)


In [125]:
batch_runner.get_model_vars_dataframe()

,p,grid_shape,Run,Burnt,Max cluster size
0,0.0,"(20, 20)",0,1,1
1,0.0,"(20, 20)",1,1,1
2,0.0,"(20, 20)",2,1,1
3,0.0,"(20, 20)",3,1,1
4,0.0,"(20, 20)",4,1,1
5,0.0,"(50, 50)",5,1,1
6,0.0,"(50, 50)",6,1,1
7,0.0,"(50, 50)",7,1,1
8,0.0,"(50, 50)",8,1,1
9,0.0,"(50, 50)",9,1,1
